In [1]:
import sklearn
import numpy as np
import pandas as pd
import re
# nltk
import nltk
import pickle

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


In [4]:
# check if there are any missing values
df.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [5]:
#install tweet-preprocessor to clean tweets
!pip install tweet-preprocessor

In [6]:
# remove special characters using the regular expression library
import re

#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [7]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [8]:
# clean training data
train_tweet = clean_tweets(df["tweet"])
train_tweet = pd.DataFrame(train_tweet)

In [9]:
# append cleaned tweets to the training data
df["clean_tweet"] = train_tweet

# compare the cleaned and uncleaned tweets
df.head(10)

,id,label,tweet,clean_tweet
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for credit i cant use cause they dont o...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,i love u take with u all the time in ur
4,5,0,factsguide: society now #motivation,factsguide society now
5,6,0,[2/2] huge fan fare and big talking before the...,2 2 huge fan fare and big talking before they ...
6,7,0,@user camping tomorrow @user @user @user @use...,camping tomorrow danny
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams can...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land
9,10,0,@user @user welcome here ! i'm it's so #gr...,welcome here im its so


In [10]:
from sklearn.model_selection import train_test_split

# extract the labels from the train data
y = df.label.values
# use 70% for the training and 30% for the test
x_train, x_test, y_train, y_test = train_test_split(df.clean_tweet.values, y,stratify=y, 
                                                    random_state=1, 
                                                    test_size=0.3, shuffle=True)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=12000)
vectoriser.fit(x_train)
print(f'Vectoriser fitted.')
print('No. of feature_words: ', len(vectoriser.get_feature_names()))

Vectoriser fitted.
No. of feature_words:  12000


c:\Users\sansk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [12]:
x_train = vectoriser.transform(x_train)
x_test  = vectoriser.transform(x_test)

In [13]:
from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(x_train, y_train)

LinearSVC()

In [14]:
model.predict(vectoriser.transform(['black']))

array([1], dtype=int64)

In [15]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, model.predict(x_test)) * 100, '%')

Accuracy score for SVC is:  95.27583689644385 %


In [16]:
with open('Model.pkl','wb') as f:
    pickle.dump(model,f)

In [19]:
with open('Model.pkl','rb') as f:
    mp = pickle.load(f)

In [23]:
z = mp.predict(vectoriser.transform(['anti-white']))

In [24]:
type(z)

numpy.ndarray

In [22]:
file = open('vectoriser-ngram-(1,2).pkl','wb')
pickle.dump(vectoriser, file)
file.close()